- Initializing

In [1]:
import numpy as np
#import matplotlib.pyplot as plt
import pandas as pd
#import seaborn as sns
#import statsmodels.api as sm
#from sklearn.linear_model import LogisticRegression
from sklearn import ensemble

- Reading data

In [2]:
df = pd.read_csv('train.csv')

- Feature engineering

In [3]:
sel_width = df.shape[0]
features = pd.get_dummies(df['label'].loc[:sel_width-1])

In [4]:
x = df.iloc[:sel_width,1:]
y = features.iloc[:,:10]

- Dataset splitting. For testing, splitting the training set to two subsets. For the final training, taking all the data.

In [5]:
x_train = x
y_train = y

In [6]:
#x_train, y_train = x[x.index % 10 != 0], y[y.index % 10 != 0]
#x_test, y_test = x[x.index % 10 == 0], y[y.index % 10 == 0]

In [7]:
x_train_orig_w = x_train.shape[1]
y_train_orig_w = y_train.shape[1]

- Random Forest Classifier

In [8]:
rfc = ensemble.RandomForestClassifier(max_depth=150, n_estimators=3000, criterion='entropy')
rfc.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=150, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=3000, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

Accuracy on a training subset

In [9]:
rfc.score(x_train,y_train)

1.0

Accuracy on a testing subset

In [11]:
rfc.score(x_test.iloc[:,:x_train_orig_w],y_test.iloc[:,:y_train_orig_w])

Storing the predicted values to array

In [13]:
z = rfc.predict(x_test.iloc[:,:x_train_orig_w])

Looking how many samples was not recognized

In [ ]:
zcnt = 0
for sis in range(0,z.shape[0]):
    if z[sis].any() != 1:
        zcnt += 1
print(zcnt)

Storing indexes of unrecognized samples

In [ ]:
x_test['notdetected'] = 0
y_test['notdetected'] = 0
x_test['nd_ind'] = 0
y_test['nd_ind'] = 0
for sis in range(0,z.shape[0]):
    if sis %300 == 0:
        print(sis, ' out of ', z.shape[0])
    if z[sis].any() != 1:
        x_test['notdetected'][sis] = 1
        x_test['nd_ind'][sis] = sis
        y_test['notdetected'][sis] = 1
        y_test['nd_ind'][sis] = sis

Creating an additional dataset out of unrecognized samples

In [ ]:
x_test_add = x_test[x_test.notdetected == 1]
y_test_add = y_test[y_test.notdetected == 1]

KNN classifier

In [14]:
from sklearn.neighbors import KNeighborsClassifier
neighbors = KNeighborsClassifier(n_neighbors=1)
neighbors.fit(x_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=1, p=2,
           weights='uniform')

Accuracy on a training subset

In [15]:
neighbors.score(x_train, y_train)

1.0

Accuracy on a testing subset

In [ ]:
neighbors.score(x_test.iloc[:,:x_train_orig_w], y_test.iloc[:,:y_train_orig_w])

Accuracy on unrecognized by RF subset

In [ ]:
neighbors.score(x_test_add.iloc[:,:x_train_orig_w], y_test_add.iloc[:,:y_train_orig_w])

Storing predicted values to array

In [ ]:
z_add = neighbors.predict(x_test_add.iloc[:,:x_train_orig_w])

Checking for unrecognized

In [ ]:
zcnt = 0
for sis in range(0,z_add.shape[0]):
    if z_add[sis].any() != 1:
        zcnt += 1
print(zcnt)

Replacing initially unrecognized values with predicted by KNN ones

In [ ]:
for sis in range(0,z_add.shape[0]):
#    if z[sis][0] == 1:
        print(sis, y_test_add.iloc[sis,y_train_orig_w+1])
        z[y_test_add.iloc[sis,y_train_orig_w+1]] = z_add[sis]



Checking for accuracy by digits and total

In [ ]:
tcnt = 0
for ppos in range(0,10):
    zcnt = 0
#    print('#',ppos)
    for sis in range(0,z.shape[0]):
        if y_test.iloc[sis,ppos] and z[sis][ppos] != y_test.iloc[sis,ppos]:
            zcnt += 1
            for pis in range(0,10):
                if z[sis][pis]:
                    print('   ', ppos, ' messed with ', pis)
    print('#', ppos, 'errors: ', zcnt)
    tcnt += zcnt
print('Total errors: ', tcnt)



In [ ]:
zcnt = 0
for sis in range(0,z.shape[0]):
    for ppos in range(0,10):
        if y_test.iloc[sis,ppos] and z[sis][ppos] != y_test.iloc[sis,ppos]:
            zcnt += 1
print(zcnt)

Calculating accuracy

In [ ]:
print('Accuracy:', (1-zcnt/x_test.shape[0])*100, '%')

Taking the testing dataset

In [24]:
df_test = pd.read_csv('test.csv')

Feature engineering

In [25]:
sel_width = 28000
#features_test = pd.DataFrame()
features_test = df_test

In [18]:
#features_test['0'] = df_test.pixel0

In [26]:
features_orig_w = features_test.shape[1]

Random forest classifier

In [27]:
z_test = rfc.predict(features_test.iloc[:,:])

Looking how many digits wasn't recognized

In [28]:
zcnt = 0
for sis in range(0,z_test.shape[0]):
    if z_test[sis].any() != 1:
        zcnt += 1
print(zcnt)

2696


Storing indexes of unrecognized samples

In [29]:
features_test['notdetected'] = 0
features_test['nd_ind'] = 0
for sis in range(0,z_test.shape[0]):
    if z_test[sis].any() != 1:
        features_test['notdetected'][sis] = 1
        features_test['nd_ind'][sis] = sis

Creating additional subset for KNN

In [30]:
features_test_add = features_test[features_test.notdetected == 1]

KNN classifier

In [31]:
z_test_add = neighbors.predict(features_test_add.iloc[:,:features_orig_w])

Checking for unrecognized samples

In [33]:
zcnt = 0
for sis in range(0,z_test_add.shape[0]):
    if z_test_add[sis].any() != 1:
        zcnt += 1
print(zcnt)

0


Replacing unlabeled data with labels, derived from KNN

In [34]:
for sis in range(0,z_test_add.shape[0]):
        z_test[features_test_add.iloc[sis,features_orig_w+1]] = z_test_add[sis]

Creating the submission dataset

In [35]:
submission = pd.DataFrame()
submission['0'] = df_test.pixel0
submission['ImageId'] = 0
submission['Label'] = 0

In [36]:
for sis in range(0,z_test.shape[0]):
    if sis%1000 == 0:
        print(sis)
    for pis in range(0,10):
        if z_test[sis][pis] == 1:
            submission.iloc[sis,1] = sis+1
            submission.iloc[sis,2] = pis

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000


In [37]:
submission.shape

(28000, 3)

In [38]:
submission.iloc[:,1:].to_csv('submission_RF_KNN.csv', index=False)

Checking SVM classifier

In [39]:
from sklearn.svm import SVC


Retaking the data

In [40]:
x = df.iloc[:sel_width,1:]
y = df.iloc[:sel_width,0]

In [41]:
#x_train, y_train = x[x.index % 10 != 0], y[y.index % 10 != 0]
#x_test, y_test = x[x.index % 10 == 0], y[y.index % 10 == 0]

In [42]:
x_train = x
y_train = y

SVM classifier

In [43]:
svm = SVC(gamma='scale')
svm.fit(x_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [ ]:
svm.score(x_train, y_train)

In [ ]:
svm.score(x_test, y_test)

Predicting unrecognized samples from RF

In [44]:
z_svm_add = svm.predict(features_test_add.iloc[:,:features_orig_w])

In [45]:
submission_svm = pd.DataFrame()
submission_svm['0'] = df_test.pixel0
submission_svm['ImageId'] = 0
submission_svm['Label'] = 0

In [46]:
for sis in range(0,z_test.shape[0]):
    for pis in range(0,10):
        if z_test[sis][pis] == 1:
            submission_svm.iloc[sis,1] = sis+1
            submission_svm.iloc[sis,2] = pis

In [47]:
for sis in range(0,z_svm_add.shape[0]):
    submission_svm.iloc[features_test_add.iloc[sis,features_orig_w+1],2] = z_svm_add[sis]

In [48]:
submission_svm.iloc[:,1:].to_csv('submission_RF_SVM.csv', index=False)

Checking pure SVM classifier

In [53]:
z_svm_sub = svm.predict(df_test.iloc[:,:features_orig_w])

In [54]:
submission_svm_clear = pd.DataFrame()
submission_svm_clear['0'] = df_test.pixel0
submission_svm_clear['ImageId'] = 0
submission_svm_clear['Label'] = 0

In [55]:
for sis in range(0,z_svm_sub.shape[0]):
    submission_svm_clear.iloc[sis,1] = sis+1
    submission_svm_clear.iloc[sis,2] = z_svm_sub[sis]

In [56]:
submission_svm_clear.iloc[:,1:].to_csv('submission_SVM.csv', index=False)

Trying to fit KNN with the data from training set AND already recognized by RF data from testing set

In [107]:
z_t = pd.DataFrame()
z_t['r0'] = features_test[features_test.notdetected==0].iloc[:,:-4].pixel0
z_t['r0'] = 0
z_t['r1'] = 0
z_t['r2'] = 0
z_t['r3'] = 0
z_t['r4'] = 0
z_t['r5'] = 0
z_t['r6'] = 0
z_t['r7'] = 0
z_t['r8'] = 0
z_t['r9'] = 0

In [108]:
ztpos = 0
for sis in range(0,z_test.shape[0]):
    if sis%3000 == 0:
        print(sis)
    if features_test['notdetected'][sis] == 0:
        for ppos in range(0,10):
            if z_test[sis][ppos]:
                z_t.iloc[ztpos,ppos] = 1        
#                z_t.iloc[ztpos,:] = z_test[sis]
        ztpos += 1


0
3000
6000
9000
12000
15000
18000
21000
24000
27000


In [158]:
x_train = df.iloc[:42000,1:]

In [159]:
m_x_train = pd.concat([x_train, features_test[features_test.notdetected==0].iloc[:,:-2]], ignore_index=True, sort=False)

In [160]:
y_train = features.iloc[:,:10]

In [161]:
y_train.columns=['r0','r1','r2','r3','r4','r5','r6','r7','r8','r9']

In [163]:
m_y_train = y_train.append([z_t], ignore_index=True)

In [164]:
m_neighbors = KNeighborsClassifier(n_neighbors=1)
m_neighbors.fit(m_x_train, m_y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=1, p=2,
           weights='uniform')

In [165]:
zm_test_add = m_neighbors.predict(features_test_add.iloc[:,:features_orig_w])

In [169]:
for sis in range(0,zm_test_add.shape[0]):
        z_test[features_test_add.iloc[sis,features_orig_w+1]] = zm_test_add[sis]

In [170]:
submission_zm = pd.DataFrame()
submission_zm['0'] = df_test.pixel0
submission_zm['ImageId'] = 0
submission_zm['Label'] = 0

In [171]:
for sis in range(0,z_test.shape[0]):
    for pis in range(0,10):
        if z_test[sis][pis] == 1:
            submission_zm.iloc[sis,1] = sis+1
            submission_zm.iloc[sis,2] = pis

In [172]:
submission_zm.shape

(28000, 3)

In [173]:
submission_zm.iloc[:,1:].to_csv('submission_RF_RF_KNN.csv', index=False)

Trying to fit RF! with the data from training set AND already recognized by RF data from testing set

In [174]:
rfc_loop = ensemble.RandomForestClassifier(max_depth=150, n_estimators=3000, criterion='entropy')
rfc_loop.fit(m_x_train,m_y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=150, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=3000, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [178]:
z_test_loop = rfc_loop.predict(features_test.iloc[:,:features_orig_w])

In [181]:
zcnt = 0
for sis in range(0,z_test_loop.shape[0]):
    if z_test_loop[sis].any() != 1:
        zcnt += 1
print(zcnt)

2289


Storing indexes of unrecognized samples

In [189]:
features_test['notdetected'] = 0
features_test['nd_ind'] = 0
for sis in range(0,z_test_loop.shape[0]):
    if sis%1000 == 0:
        print(sis)
    if z_test_loop[sis].any() != 1:
        features_test['notdetected'][sis] = 1
        features_test['nd_ind'][sis] = sis


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000


Creating additional subset

In [191]:
features_test_add = features_test[features_test.notdetected == 1]

KNN classifier

In [193]:
zmk_test_add = m_neighbors.predict(features_test_add.iloc[:,:features_orig_w])

Checking for unrecognized samples

In [196]:
zcnt = 0
for sis in range(0,zmk_test_add.shape[0]):
    if zmk_test_add[sis].any() != 1:
        zcnt += 1
print(zcnt)

0


Replacing unlabeled data with labels, derived from KNN

In [197]:
for sis in range(0,zmk_test_add.shape[0]):
        z_test_loop[features_test_add.iloc[sis,features_orig_w+1]] = zmk_test_add[sis]

Creating the submission dataset

In [198]:
submission_zmk = pd.DataFrame()
submission_zmk['0'] = df_test.pixel0
submission_zmk['ImageId'] = 0
submission_zmk['Label'] = 0

In [199]:
for sis in range(0,z_test_loop.shape[0]):
    if sis%1000 == 0:
        print(sis)
    for pis in range(0,10):
        if z_test_loop[sis][pis] == 1:
            submission_zmk.iloc[sis,1] = sis+1
            submission_zmk.iloc[sis,2] = pis

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000


In [200]:
submission_zmk.shape

(28000, 3)

In [201]:
submission_zmk.iloc[:,1:].to_csv('submission_RF_KNN_LOOP.csv', index=False)

Trying to fit SVM with the data from training set AND already recognized by RF data from testing set

In [203]:
m_y_train['label'] = 0
for sis in range(0,m_y_train.shape[0]):
    if sis%1000 == 0:
        print(sis)
    for pis in range(0,10):
        if m_y_train['r'+str(pis)][sis] == 1:
            m_y_train['label'][sis] = pis

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000


In [206]:
svm_loop = SVC(gamma='scale')
svm_loop.fit(m_x_train, m_y_train['label'])

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [217]:
z_svm_loop = svm_loop.predict(df_test.iloc[:,:features_orig_w])

In [218]:
submission_svm_loop = pd.DataFrame()
submission_svm_loop['0'] = df_test.pixel0
submission_svm_loop['ImageId'] = 0
submission_svm_loop['Label'] = 0

In [219]:
for sis in range(0,z_svm_loop.shape[0]):
    submission_svm_loop['ImageId'][sis] = sis+1
    submission_svm_loop['Label'][sis] = z_svm_loop[sis]

In [220]:
submission_svm_loop.shape

(28000, 3)

In [221]:
submission_svm_loop.iloc[:,1:].to_csv('submission_SVM_LOOP.csv', index=False)

Trying to fit SVM with the data from training set AND already recognized by RF data from testing set and then to use substitute unrecognized RF data with ones, predicted by the latest overfitted SVM

In [222]:
z_svm_loop_add = svm_loop.predict(features_test_add.iloc[:,:features_orig_w])

In [225]:
submission_svm_add_loop = pd.DataFrame()
submission_svm_add_loop['0'] = df_test.pixel0
submission_svm_add_loop['ImageId'] = 0
submission_svm_add_loop['Label'] = 0

In [227]:
for sis in range(0,z_test_loop.shape[0]):
    for pis in range(0,10):
        if z_test_loop[sis][pis] == 1:
            submission_svm_add_loop['ImageId'][sis] = sis+1
            submission_svm_add_loop['Label'][sis] = pis

In [229]:
for sis in range(0,z_svm_loop_add.shape[0]):
    submission_svm_add_loop.iloc[features_test_add.iloc[sis,features_orig_w+1],2] = z_svm_loop_add[sis]

In [230]:
submission_svm_add_loop.iloc[:,1:].to_csv('submission_RF_SVM_LOOP.csv', index=False)

That's it...